In [1]:
# -*- coding: cp1252 -*-


from mechanize import Browser
import os
import pandas as pd
import html5lib
from bs4 import BeautifulSoup  


#os.chdir(r"E:\Dropbox\Dropbox\0_DB\A_Elaborazioni\Coeweb")

class coeweb(object):
    
    def __init__(self, link):
        self.pandas_db = pd.DataFrame()
        self.link = link
        br = Browser()
        br.open(self.link)
        br.select_form(nr=0)
        
        self.paesi = []
        self.paesi_descizione = []
        control = br.form.find_control("PAESE")
        if control.type == "select":  
            for item in control.items:
                self.paesi.append(item.name)
                self.paesi_descizione.append((item.name, str([label.text  for label in item.get_labels()])))
                
        self.territori = []
        self.territori_descrizione = []
        control = br.form.find_control("CODTERRITORIO")
        if control.type == "select":  
            for item in control.items:
                self.territori.append((item.name, str([label.text  for label in item.get_labels()])))
                self.territori_descrizione.append((item.name, str([label.text  for label in item.get_labels()])))
                
        self.anni = []
        control = br.form.find_control("ANNO")
        if control.type == "select":  
            for item in control.items:
                self.anni.append((item.name, str([label.text  for label in item.get_labels()])))
        self.mese = []
        control = br.form.find_control("MESE")
        if control.type == "select": 
            for item in control.items:
                self.mese.append((item.name, str([label.text  for label in item.get_labels()])))
        self.cumulato = []
        control = br.form.find_control("TIPOVALORE")
        for item in control.items:
            self.cumulato.append((item.name, str([label.text  for label in item.get_labels()])))
        
        self.bric = ['508', '720', '664', '75']
        #[('508', "['508 - Brasile']"), ('720', "['720 - Cina']"), ('664', "['664 - India']"), ('75', "['75 - Russia']")]
    
    def convert_string_euro(self,x):
        '''questa funzione rimuove la formattazione dei numeri di coeweb, trasformando il testo in numeri'''
        try: 
            return float(x.replace(".", ""))
        except:
            pass
    

    def scarica_uno(self, territorio = ["309048"], paese = ["1033"], cumulato = ['M'], anno = ["2013"], mese = ["4"]  ):# , paese, anno, trimestre, modalita)
        
        br = Browser()
        br.open(self.link)

        br.select_form(nr=0)

        control_territorio = br.form.find_control("CODTERRITORIO")
        br[control_territorio.name] = territorio #["309048"]

        control_paese = br.form.find_control("PAESE")
        br[control_paese.name] = paese #["309048"]

        control_anno = br.form.find_control("ANNO")
        br[control_anno.name] = anno #["309048"]

        control_mese = br.form.find_control("MESE")
        br[control_mese.name] = mese #["309048"]

        control_cumulato = br.form.find_control("TIPOVALORE")
        br[control_cumulato.name] = cumulato #["309048"]

        nome_stringa = territorio[0] + "_" + paese[0] + "_" + anno[0] + "_" + mese[0] + "_" + cumulato[0]


        response = br.submit()
        out_file = open(os.path.join(os.getcwd(), "scaricati", nome_stringa +".html"), "w")
        out_file.write(response.read()) 
        out_file.close()
        #os.system("start " + nome_stringa +".html")
        self.dati = pd.read_html(os.path.join(os.getcwd(), "scaricati", nome_stringa +".html"), header= 0, index_col=0, encoding = "cp1252")#'utf8'
        #os.remove(territorio[0] +".html")
        print "download_" + nome_stringa
        br.close()
        return self.dati
        
        # i dati dovrebbero essere raccolti in un pandas 
        
    def scarica_molti(self, territorio = ["309048"],   lista_paesi = None, lista_anni = None, lista_mesi = None, lista_cumulato = ["M"]):
        ''' questa funzione permette di scaricare per uno o più territori uno o più settori, per uno o più periodi di tempo'''
        #self.pandas_db = pd.DataFrame()
        
        def nome_paese(dati, codice):
            for item in dati:
                if item[0] == codice:
                    pulito1= item[1].split("'",)[1]
                    pulito2 = pulito1.split(" - ",)[1]
                    print pulito2
                    return pulito2
        
        def nome_territorio(dati, codice):
            for item in dati:
                if item[0] == codice:
                    try:
                        pulito1= item[1].split("'")[1]
                        pulito1 = pulito1.replace("-","")
                        pulito1 = pulito1.replace("'","")
                        return pulito1
                    except:
                        pass
        
        
        for item_territori in territorio:
            for item_paesi in lista_paesi:
                for item_anni in lista_anni:
                    for item_mesi in lista_mesi:

                        dataset =  self.scarica_uno([item_territori], paese = [str(item_paesi)],  anno = [str(item_anni)], mese = [str(item_mesi)], cumulato = lista_cumulato)
                        dataset = dataset[1]#.set_index(territorio, str(item_paesi), str(item_anni), str(item_mesi)])
                        
                        
                        dataset["territorio"] = nome_territorio(self.territori_descrizione,  item_territori)  
                        #dataset["territorio"] = item_territori
                        
                        #nome_paese(self.paesi_descizione, str(item_paesi))
                        dataset["paese"] = nome_paese(self.paesi_descizione, str(item_paesi))
                        dataset["anno"] = str(item_anni)
                        dataset["mese"] = str(item_mesi)
                        dataset["cumulato"] = lista_cumulato[0]
                        dataset.set_index(["territorio","paese","anno","mese","cumulato"], append = True, inplace = True)
                        dataset["DATI EXPORT"] = dataset["EXP" + str(item_anni)]
                          
                        
                        
                        dataset = dataset["DATI EXPORT"].apply(lambda x: self.convert_string_euro(x))
                        
                        
                        
                        # datasetcompleto
                        self.pandas_db = pd.concat([self.pandas_db, pd.DataFrame(dataset)])
                        # settori ateco disponibili
                        self.pandas_db.settori = set(self.pandas_db.reset_index()["Divisioni"])
                        
                            #print "errore_" + territorio[0] + "_" + str(item_paesi) + "_" + str(item_anni) + "_" + str(item_mesi) + "_" + lista_cumulato[0]

        for scaricato in os.listdir(os.path.join(os.getcwd(), "scaricati")):
            os.remove(os.path.join(os.getcwd(), "scaricati", scaricato))
        

        print "eseguito"
    
    
    
    def analizza_settori(self, settori, interpolazione = "no"):
        ''' questa funzione restituisce un dataframe e un dizionario contenente le serie storiche per i settori inseriti come argomento [lista]. 
        Effettua anche l'interpolazione dei dati mancanti'''
        self.dict_settore = {}
        for item in settori:
            dati_settore = self.pandas_db.xs(item, level=0)
            if interpolazione == "yes":
                dati_settore["DATI EXPORT"] = dati_settore["DATI EXPORT"].apply(lambda x: convert_string_euro(x)).interpolate()
            else:
                dati_settore["DATI EXPORT"] = dati_settore["DATI EXPORT"].apply(lambda x: convert_string_euro(x))
            self.dict_settore[item] = dati_settore["DATI EXPORT"]
        self.dataframe_settori = pd.DataFrame(self.dict_settore)
        return self.dataframe_settori

In [2]:
def convert_string_euro(x):
    try: 
        return float(x.replace(".", ""))
    except:
        pass

In [3]:
os.getcwd()

'e:\\Dropbox\\Dropbox\\0_DB\\coeweb'

In [22]:

lista_anni = [str(i) for i in range(2009,2016)]


In [97]:
lista_anni

['2009', '2010', '2011', '2012', '2013', '2014', '2015']

In [4]:
Firenze = coeweb("http://www.coeweb.istat.it/predefinite/tutto_paese_merce.asp?livello=ATE07_AT2&riga=MERCE&territorio=S&AG=S")

In [10]:
Firenze.paesi_descizione

[('1', "['1 - Francia']"),
 ('2', "['2 - Belgio e Lussemburgo']"),
 ('3', "['3 - Paesi Bassi']"),
 ('4', "['4 - Germania']"),
 ('6', "['6 - Regno Unito']"),
 ('7', "['7 - Irlanda']"),
 ('8', "['8 - Danimarca']"),
 ('9', "['9 - Grecia']"),
 ('10', "['10 - Portogallo']"),
 ('11', "['11 - Spagna']"),
 ('17', "['17 - Belgio']"),
 ('18', "['18 - Lussemburgo']"),
 ('21', "['21 - Ceuta']"),
 ('22', "['22 - Ceuta e Melilla']"),
 ('23', "['23 - Melilla']"),
 ('24', "['24 - Islanda']"),
 ('25', "['25 - Isole F\\xe6r\\xf8er']"),
 ('27', "['27 - Svalbard (arcipelago delle)']"),
 ('28', "['28 - Norvegia']"),
 ('30', "['30 - Svezia']"),
 ('32', "['32 - Finlandia']"),
 ('37', "['37 - Liechtenstein']"),
 ('38', "['38 - Austria']"),
 ('39', "['39 - Svizzera']"),
 ('41', "['41 - Faer \\xd8er']"),
 ('43', "['43 - Andorra']"),
 ('44', "['44 - Gibilterra']"),
 ('45', "['45 - Santa Sede (Stato della Citt\\xe0 del Vaticano)']"),
 ('46', "['46 - Malta']"),
 ('47', "['47 - San Marino']"),
 ('48', "['48 - Iugos

In [24]:
metropolitana = [('309', "['---Toscana']"),
 ('309047', "['------Pistoia']"),
 ('309048', "['------Firenze']"),
 ('309100', "['------Prato']")]

In [25]:
CodiciMetropolitana = []
for item in metropolitana:
    CodiciMetropolitana.append(item[0])

In [90]:
CodiciMetropolitana

['309', '309047', '309048', '309100']

In [26]:
Firenze.scarica_molti(territorio = CodiciMetropolitana, lista_paesi = ['6'], lista_anni = lista_anni, lista_mesi = ["9"], lista_cumulato = ['C'])

download_309_6_2009_9_C
Regno Unito
download_309_6_2010_9_C
Regno Unito
download_309_6_2011_9_C
Regno Unito
download_309_6_2012_9_C
Regno Unito
download_309_6_2013_9_C
Regno Unito
download_309_6_2014_9_C
Regno Unito
download_309_6_2015_9_C
Regno Unito
download_309047_6_2009_9_C
Regno Unito
download_309047_6_2010_9_C
Regno Unito
download_309047_6_2011_9_C
Regno Unito
download_309047_6_2012_9_C
Regno Unito
download_309047_6_2013_9_C
Regno Unito
download_309047_6_2014_9_C
Regno Unito
download_309047_6_2015_9_C
Regno Unito
download_309048_6_2009_9_C
Regno Unito
download_309048_6_2010_9_C
Regno Unito
download_309048_6_2011_9_C
Regno Unito
download_309048_6_2012_9_C
Regno Unito
download_309048_6_2013_9_C
Regno Unito
download_309048_6_2014_9_C
Regno Unito
download_309048_6_2015_9_C
Regno Unito
download_309100_6_2009_9_C
Regno Unito
download_309100_6_2010_9_C
Regno Unito
download_309100_6_2011_9_C
Regno Unito
download_309100_6_2012_9_C
Regno Unito
download_309100_6_2013_9_C
Regno Unito
downloa

In [179]:
dati_toscana_completi = Firenze.pandas_db

In [172]:
dati_toscana_completi = dati_toscana_completi.reset_index()

In [173]:
dati_toscana_completi["anno"] = pd.to_datetime(dati_toscana_completi.anno, format="%Y")

In [174]:
dati_toscana_completi.drop(["paese", "mese", "cumulato"], axis = 1, inplace = True)

In [88]:
dati_toscana_completi.to_excel("dati_metropolitana_completi.xlsx")

In [177]:
dati_toscana_completi

,Divisioni,territorio,anno,DATI EXPORT
0,"AA01-Prodotti agricoli, animali e della caccia",Toscana,2009-01-01,19002036.0
1,AA02-Prodotti della silvicoltura,Toscana,2009-01-01,674892.0
2,AA03-Prodotti della pesca e dell'acquacoltura,Toscana,2009-01-01,2301.0
3,BB05-Carbone (esclusa torba),Toscana,2009-01-01,0.0
4,BB06-Petrolio greggio e gas naturale,Toscana,2009-01-01,0.0
5,BB07-Minerali metalliferi,Toscana,2009-01-01,386.0
6,BB08-Altri minerali da cave e miniere,Toscana,2009-01-01,997726.0
7,CA10-Prodotti alimentari,Toscana,2009-01-01,82953860.0
8,CA11-Bevande,Toscana,2009-01-01,27951264.0
9,CA12-Tabacco,Toscana,2009-01-01,19989.0


In [175]:
Firenze

In [181]:
dati_toscana_completi[dati_toscana_completi.index.get_level_values('Divisioni').isin(['Totale'])].unstack("anno")

DATI EXPORT                \
anno                                                    2009          2010   
Divisioni territorio paese       mese cumulato                               
Totale    Firenze    Regno Unito 9    C         2.901558e+08  3.623726e+08   
          Pistoia    Regno Unito 9    C         8.531646e+07  9.995966e+07   
          Prato      Regno Unito 9    C         8.213081e+07  1.143968e+08   
          Toscana    Regno Unito 9    C         1.528747e+09  1.473169e+09   

                                                                            \
anno                                                    2011          2012   
Divisioni territorio paese       mese cumulato                               
Totale    Firenze    Regno Unito 9    C         3.356670e+08  3.613749e+08   
          Pistoia    Regno Unito 9    C         1.045361e+08  9.753491e+07   
          Prato      Regno Unito 9    C         1.087060e+08  9.060144e+07   
          Toscana    Regno Unito 9    C         1.519613e+09  1.486383e+09   

                                                                            \
anno                                                    2013          2014   
Divisioni territorio paese       mese cumulato                               
Totale    Firenze    Regno Unito 9    C         5.123522e+08  5.352489e+08   
          Pistoia    Regno Unito 9    C         1.052287e+08  1.108157e+08   
          Prato      Regno Unito 9    C         9.367338e+07  1.115087e+08   
          Toscana    Regno Unito 9    C         1.615316e+09  1.686907e+09   

                                                              
anno                                                    2015  
Divisioni territorio paese       mese cumulato                
Totale    Firenze    Regno Unito 9    C         5.816399e+08  
          Pistoia    Regno Unito 9    C         1.242330e+08  
          Prato      Regno Unito 9    C         1.427064e+08  
          Toscana    Regno Unito 9    C         1.802210e+09

In [41]:
#["2015"]

Divisioni  territorio  paese        mese  cumulato
Totale     Firenze     Regno Unito  9     C           5.816399e+08
           Pistoia     Regno Unito  9     C           1.242330e+08
           Prato       Regno Unito  9     C           1.427064e+08
           Toscana     Regno Unito  9     C           1.802210e+09
Name: 2015, dtype: float64

In [220]:
%matplotlib inline

(serie_storica["DATI EXPORT"].divide(serie_storica["DATI EXPORT"]["2009"], axis = 0) * 100).to_excel("andamento_export_totale_numero_indice.xlsx")

In [219]:
serie_storica["DATI EXPORT"].to_excel("andamento_export_totale.xlsx")

In [184]:
dati_toscana_completi

,,,,,,DATI EXPORT
Divisioni,territorio,paese,anno,mese,cumulato,
"AA01-Prodotti agricoli, animali e della caccia",Toscana,Regno Unito,2009,9,C,19002036.0
AA02-Prodotti della silvicoltura,Toscana,Regno Unito,2009,9,C,674892.0
AA03-Prodotti della pesca e dell'acquacoltura,Toscana,Regno Unito,2009,9,C,2301.0
BB05-Carbone (esclusa torba),Toscana,Regno Unito,2009,9,C,0.0
BB06-Petrolio greggio e gas naturale,Toscana,Regno Unito,2009,9,C,0.0
BB07-Minerali metalliferi,Toscana,Regno Unito,2009,9,C,386.0
BB08-Altri minerali da cave e miniere,Toscana,Regno Unito,2009,9,C,997726.0
CA10-Prodotti alimentari,Toscana,Regno Unito,2009,9,C,82953860.0
CA11-Bevande,Toscana,Regno Unito,2009,9,C,27951264.0


In [192]:
moda = ["CB13-Prodotti tessili", 
 u'CB14-Articoli di abbigliamento (anche in pelle e in pelliccia)',
 u'CB15-Articoli in pelle (escluso abbigliamento) e simili']

serie_storica_moda = dati_toscana_completi[dati_toscana_completi.index.get_level_values('Divisioni').isin(moda)]

In [194]:
pd.options.display.float_format = '€{:,.2f}'.format
serie_storica_moda = serie_storica_moda.unstack("anno")

In [216]:
serie_storica_moda.to_excel("andamento_moda.xlsx")

In [217]:
pd.options.display.float_format = '{:,.2f}'.format
(serie_storica_moda["DATI EXPORT"].divide(serie_storica_moda["DATI EXPORT"]["2009"], axis = 0) * 100).to_excel("andamento_moda_numeri_indice.xlsx")

In [11]:
Italia = coeweb("http://www.coeweb.istat.it/predefinite/tutto_paese_merce.asp?livello=ATE07_AT2&riga=MERCE&territorio=S&AG=S")

In [12]:
italia_province = []
for item in Italia.territori:
    if len(item[0]) == 6:
        italia_province.append(item[0])

In [13]:
#italia_province

In [101]:
Italia.scarica_molti(territorio = italia_province, lista_paesi = ['6'], lista_anni = ["2015"], lista_mesi = ["9"], lista_cumulato = ['C'])

download_101001_6_2015_9_C
Regno Unito
download_101002_6_2015_9_C
Regno Unito
download_101003_6_2015_9_C
Regno Unito
download_101004_6_2015_9_C
Regno Unito
download_101005_6_2015_9_C
Regno Unito
download_101006_6_2015_9_C
Regno Unito
download_101096_6_2015_9_C
Regno Unito
download_101103_6_2015_9_C
Regno Unito
download_102007_6_2015_9_C
Regno Unito
download_103012_6_2015_9_C
Regno Unito
download_103013_6_2015_9_C
Regno Unito
download_103014_6_2015_9_C
Regno Unito
download_103015_6_2015_9_C
Regno Unito
download_103016_6_2015_9_C
Regno Unito
download_103017_6_2015_9_C
Regno Unito
download_103018_6_2015_9_C
Regno Unito
download_103019_6_2015_9_C
Regno Unito
download_103020_6_2015_9_C
Regno Unito
download_103097_6_2015_9_C
Regno Unito
download_103098_6_2015_9_C
Regno Unito
download_103108_6_2015_9_C
Regno Unito
download_107008_6_2015_9_C
Regno Unito
download_107009_6_2015_9_C
Regno Unito
download_107010_6_2015_9_C
Regno Unito
download_107011_6_2015_9_C
Regno Unito
download_204021_6_2015_9_

In [103]:
Italia_df = Italia.pandas_db

In [106]:
Italia_df = Italia_df.reset_index()

In [141]:
key = pd.read_excel("aggancio_nuts_ok.xlsx")

In [143]:
key

,COEWEB,NUTS,NUTS_MAP,EUROSTAT
0,Torino,ITC11,ITC11,Torino
1,Vercelli,ITC12,ITC12,Vercelli
2,Biella,ITC13,ITC13,Biella
3,VerbanoCusioOssola,ITC14,ITC14,Verbano-Cusio-Ossola
4,Novara,ITC15,ITC15,Novara
5,Cuneo,ITC16,ITC16,Cuneo
6,Asti,ITC17,ITC17,Asti
7,Alessandria,ITC18,ITC18,Alessandria
8,Aosta,ITC20,ITC20,Valle d'Aosta/Vallée d'Aoste
9,Imperia,ITC31,ITC31,Imperia


In [147]:
Italia_map = pd.merge(Italia_df, key, left_on = "territorio", right_on ="COEWEB")

In [148]:
Italia_map.to_excel("Italia_map.xlsx")

In [223]:
Italia_map.to_excel("classifica_provinciale.xlsx")

In [115]:
Italia_df[Italia_df["Divisioni"] == "Totale"].sort_values("DATI EXPORT", ascending = False).to_excel("classifica.xlsx")#["territorio"].unique()

In [225]:
classifica = Italia_df[Italia_df["Divisioni"] == "Totale"].sort_values("DATI EXPORT", ascending = False)

In [226]:
classifica = pd.merge(classifica, key, left_on = "territorio", right_on ="COEWEB")

In [228]:
classifica.to_excel("classifica.xlsx")

In [138]:
#Italia_df[Italia_df["Divisioni"] == "Totale"].sort_values("DATI EXPORT", ascending = False)

In [128]:
Divisioni = Italia_df.groupby("Divisioni").sum()

In [ ]:
Divisioni.drop("Totale",axis =0, inplace = True)

In [132]:
Divisioni["%"] = Divisioni["DATI EXPORT"] / Divisioni["DATI EXPORT"].sum() * 100 #

In [142]:
Italia_df

,Divisioni,territorio,paese,anno,mese,cumulato,DATI EXPORT
0,"AA01-Prodotti agricoli, animali e della caccia",Torino,Regno Unito,2015,9,C,100882.0
1,AA02-Prodotti della silvicoltura,Torino,Regno Unito,2015,9,C,0.0
2,AA03-Prodotti della pesca e dell'acquacoltura,Torino,Regno Unito,2015,9,C,15.0
3,BB06-Petrolio greggio e gas naturale,Torino,Regno Unito,2015,9,C,0.0
4,BB07-Minerali metalliferi,Torino,Regno Unito,2015,9,C,0.0
5,BB08-Altri minerali da cave e miniere,Torino,Regno Unito,2015,9,C,748679.0
6,CA10-Prodotti alimentari,Torino,Regno Unito,2015,9,C,40508394.0
7,CA11-Bevande,Torino,Regno Unito,2015,9,C,13871997.0
8,CA12-Tabacco,Torino,Regno Unito,2015,9,C,305.0
9,CB13-Prodotti tessili,Torino,Regno Unito,2015,9,C,13542354.0


In [222]:
Divisioni.sort_values("DATI EXPORT", ascending = False).to_excel("Totale_nazionale_Divisioni.xlsx")

In [14]:
#Mondo = '1033'
Italia.scarica_molti(territorio = italia_province, lista_paesi = ['6', '1033'], lista_anni = ["2015"], lista_mesi = ["9"], lista_cumulato = ['C'])

download_101001_6_2015_9_C
Regno Unito
download_101001_1033_2015_9_C
[MONDO]
download_101002_6_2015_9_C
Regno Unito
download_101002_1033_2015_9_C
[MONDO]
download_101003_6_2015_9_C
Regno Unito
download_101003_1033_2015_9_C
[MONDO]
download_101004_6_2015_9_C
Regno Unito
download_101004_1033_2015_9_C
[MONDO]
download_101005_6_2015_9_C
Regno Unito
download_101005_1033_2015_9_C
[MONDO]
download_101006_6_2015_9_C
Regno Unito
download_101006_1033_2015_9_C
[MONDO]
download_101096_6_2015_9_C
Regno Unito
download_101096_1033_2015_9_C
[MONDO]
download_101103_6_2015_9_C
Regno Unito
download_101103_1033_2015_9_C
[MONDO]
download_102007_6_2015_9_C
Regno Unito
download_102007_1033_2015_9_C
[MONDO]
download_103012_6_2015_9_C
Regno Unito
download_103012_1033_2015_9_C
[MONDO]
download_103013_6_2015_9_C
Regno Unito
download_103013_1033_2015_9_C
[MONDO]
download_103014_6_2015_9_C
Regno Unito
download_103014_1033_2015_9_C
[MONDO]
download_103015_6_2015_9_C
Regno Unito
download_103015_1033_2015_9_C
[MONDO]

In [16]:
Italia_Export_Regno_Unito_Totale = Italia.pandas_db

In [17]:
Italia_Export_Regno_Unito_Totale

,,,,,,DATI EXPORT
Divisioni,territorio,paese,anno,mese,cumulato,
"AA01-Prodotti agricoli, animali e della caccia",Torino,Regno Unito,2015,9,C,1.008820e+05
AA02-Prodotti della silvicoltura,Torino,Regno Unito,2015,9,C,0.000000e+00
AA03-Prodotti della pesca e dell'acquacoltura,Torino,Regno Unito,2015,9,C,1.500000e+01
BB06-Petrolio greggio e gas naturale,Torino,Regno Unito,2015,9,C,0.000000e+00
BB07-Minerali metalliferi,Torino,Regno Unito,2015,9,C,0.000000e+00
BB08-Altri minerali da cave e miniere,Torino,Regno Unito,2015,9,C,7.486790e+05
CA10-Prodotti alimentari,Torino,Regno Unito,2015,9,C,4.050839e+07
CA11-Bevande,Torino,Regno Unito,2015,9,C,1.387200e+07
CA12-Tabacco,Torino,Regno Unito,2015,9,C,3.050000e+02


In [18]:
#Italia_Export_Regno_Unito_Totale[{ 'paese' : ['Regno Unito'] }]

TypeError: unhashable type: 'dict'

In [20]:
Italia_Export_Regno_Unito_Totale

,,,,,,DATI EXPORT
Divisioni,territorio,paese,anno,mese,cumulato,
"AA01-Prodotti agricoli, animali e della caccia",Torino,Regno Unito,2015,9,C,1.008820e+05
AA02-Prodotti della silvicoltura,Torino,Regno Unito,2015,9,C,0.000000e+00
AA03-Prodotti della pesca e dell'acquacoltura,Torino,Regno Unito,2015,9,C,1.500000e+01
BB06-Petrolio greggio e gas naturale,Torino,Regno Unito,2015,9,C,0.000000e+00
BB07-Minerali metalliferi,Torino,Regno Unito,2015,9,C,0.000000e+00
BB08-Altri minerali da cave e miniere,Torino,Regno Unito,2015,9,C,7.486790e+05
CA10-Prodotti alimentari,Torino,Regno Unito,2015,9,C,4.050839e+07
CA11-Bevande,Torino,Regno Unito,2015,9,C,1.387200e+07
CA12-Tabacco,Torino,Regno Unito,2015,9,C,3.050000e+02


In [41]:
UK = Italia_Export_Regno_Unito_Totale.xs('Regno Unito', level='paese')

In [42]:
Mondo = Italia_Export_Regno_Unito_Totale.xs('[MONDO]', level='paese')

In [43]:
UK

,,,,,DATI EXPORT
Divisioni,territorio,anno,mese,cumulato,
"AA01-Prodotti agricoli, animali e della caccia",Torino,2015,9,C,100882.0
AA02-Prodotti della silvicoltura,Torino,2015,9,C,0.0
AA03-Prodotti della pesca e dell'acquacoltura,Torino,2015,9,C,15.0
BB06-Petrolio greggio e gas naturale,Torino,2015,9,C,0.0
BB07-Minerali metalliferi,Torino,2015,9,C,0.0
BB08-Altri minerali da cave e miniere,Torino,2015,9,C,748679.0
CA10-Prodotti alimentari,Torino,2015,9,C,40508394.0
CA11-Bevande,Torino,2015,9,C,13871997.0
CA12-Tabacco,Torino,2015,9,C,305.0


In [46]:
UK / Mondo * 100

DATI EXPORT
Divisioni                                          territorio          anno mese cumulato             
NaN                                                Agrigento           2015 9    C                 NaN
                                                   Alessandria         2015 9    C                 NaN
                                                   Ancona              2015 9    C                 NaN
                                                   Aosta               2015 9    C                 NaN
                                                   Aquila"]            2015 9    C                 NaN
                                                   Arezzo              2015 9    C                 NaN
                                                   Ascoli Piceno       2015 9    C                 NaN
                                                   Asti                2015 9    C                 NaN
                                                   Avellino            2015 9    C                 NaN
                                                   Bari                2015 9    C                 NaN
                                                   BarlettaAndriaTrani 2015 9    C                 NaN
                                                   Belluno             2015 9    C                 NaN
                                                   Benevento           2015 9    C                 NaN
                                                   Bergamo             2015 9    C                 NaN
                                                   Biella              2015 9    C                 NaN
                                                   Bologna             2015 9    C                 NaN
                                                   Bolzano/Bozen       2015 9    C                 NaN
                                                   Brescia             2015 9    C                 NaN
                                                   Brindisi            2015 9    C                 NaN
                                                   Cagliari            2015 9    C                 NaN
                                                   Caltanissetta       2015 9    C                 NaN
                                                   Campobasso          2015 9    C                 NaN
                                                   CarboniaIglesias    2015 9    C                 NaN
                                                   Caserta             2015 9    C                 NaN
                                                   Catania             2015 9    C                 NaN
                                                   Catanzaro           2015 9    C                 NaN
                                                   Chieti              2015 9    C                 NaN
                                                   Como                2015 9    C                 NaN
                                                   Cosenza             2015 9    C                 NaN
                                                   Cremona             2015 9    C                 NaN
...                                                                                                ...
VV89-Merci dichiarate come provviste di bordo, ... Ragusa              2015 9    C                 NaN
                                                   Ravenna             2015 9    C            0.000000
                                                   Reggio di Calabria  2015 9    C            0.000000
                                                   Rieti               2015 9    C                 NaN
                                                   Rimini              2015 9    C            0.000000
                                                   Roma                2015 9    C            0.000000
                                                   Rovigo              201